In [70]:
#import libraries
import os
from os import listdir
from os.path import isfile, join
from docx2python import docx2python
import pymupdf,fitz
import platform
import spacy

In [ ]:
#output json array


In [71]:
#nlp Processing
nlp = spacy.load("en_core_web_lg")
#add pipeline
ruler=nlp.add_pipe("entity_ruler",before="ner")
#ruler pipeline add pattern
skills = "skill_pattern.jsonl"
ruler.from_disk(skills)
def processTextUsingSpacy(inputText, fileName):
    mobileNumber = ''
    #re find(mobile-pattern)
    #check list empty
    #if not store mobile in variable
    doc = nlp(inputText)
    skills=set()
    required_skill={'.NET', 'java','llm'}
    detected_req_skill_count=0
    name='empty'
    email=''
    
    for ent in doc.ents:
        #print('ent.label:', ent.label_, 'text:', ent.text)
        if(ent.label_ == 'SKILL'):
            skills.add(ent.text)
        if (ent.label_ == 'EMAIL'):
            email = ent.text.lower()
            
    for ent in doc.ents:
        if (ent.label_ == 'PERSON'):
            if(len(ent.text) > 4):
                shortName = ent.text[:3].lower()
                if(shortName in email):
                    name = ent.text


    for i in required_skill:
        if i in skills:
            detected_req_skill_count = detected_req_skill_count+1
    print("------Resume NLP Output-------")
    print(name,'candidate skills:',skills)
    print('required skills:',required_skill)
    print('email:',email)
    if detected_req_skill_count == len(required_skill):
        print(name,' selected')
    else:
        print(name,' rejected')
    #create a json string
    #insert json string into array

In [68]:
#clear jsonArray
cwd = os.getcwd()
path = os.path.join(cwd, "resume")
fileList = [f for f in os.listdir(path)]
print('Total Number of resumes to be processed:',len(fileList))
for file in fileList:
    if('.docx' in file):
        # extract docx content
        wordfile = os.path.join(cwd, "resume",file)
        with docx2python(wordfile) as docx_content:
            processTextUsingSpacy(docx_content.text, file)
    if('.pdf' in file):
        pdfFile = os.path.join(cwd, "resume",file)
        doc=fitz.open(pdfFile)
        text=""
        for page in doc:
            text = text+str(page.get_text())
        tx ="".join(text.split("\n"))
        doc.close()       
        processTextUsingSpacy(tx,file)


Total Number of resumes to be processed: 2
------Resume NLP Output-------
Aswin Loganathan candidate skills: {'java', '.NET'}
required skills: {'java', 'llm', '.NET'}
email: aswin.l@pixelexpert.net
Aswin Loganathan  rejected
------Resume NLP Output-------
Jane Smith candidate skills: set()
required skills: {'java', 'llm', '.NET'}
email: janesmith@example.com
Jane Smith  rejected


In [ ]:
#Print user readable Results by parsing array of jsonString